# Assertion RLA

## Overview of the assertion audit tool

The tool requires as input:

+ audit-specific and contest-specific parameters, such as
    - whether to sample with or without replacement
    - the name of the risk function to use, and any parameters it requires
    - a risk limit for each contest to be audited
    - the social choice function for each contest, including the number of winners
    - candidate identifiers
+ a ballot manifest**
+ a random seed
+ a file of cast vote records
+ reported results for each contest
+ json files of assertions for IRV contests (one file per IRV contest)
+ human reading of voter intent from the paper cards selected for audit

** The ballot manifest could be only for cards purported to contain the
contests under audit (manifest_type == "STYLE"), or could include cards that might not contain the
contest (manifest_type == "ALL"). These are treated differently. If the sample is to be drawn only from cards that--according to the CVR--contain the contest, and a sampled card turns out not to
contain the contest, that is considered a discrepancy, dealt with using the "phantoms to zombies" approach.
It is assumed that every CVR corresponds to a card in the manifest, but there might
be cards cast in the contest for which there is no corresponding CVR. In that case,
phantom records are created to ensure that the audit is still truly risk-limiting.

Given an independent (i.e., not relying on the voting system) upper bound on the number of cards that contain the contest, if the number of CVRs that contain the contest does not exceed that bound, we can sample from paper purported to contain the contest and use the "zombies" approach (Banuelos & Stark) to deal with missing CVRs. This can greatly increase the efficiency of the audit if the contest is on only a small percentage of the cast cards.

Any sampled phantom card (i.e., a card for which there are no CVRs) is treated as if its CVR is a non-vote (which it is), and as if its MVR was least favorable (a "zombie" producing the greatest doubt in every assertion, separately). Any sampled card for which
there is a CVR is compared to its corresponding CVR. 
If the card turns out not to contain the contest (despite the fact that the CVR says it does), the MVR is treated in the least favorable way for each assertion (i.e., as a zombie rather than as a non-vote).

The tool helps select cards for audit, and reports when the audit has found sufficiently strong evidence to stop.

The tool exports a log of all the audit inputs except the CVR file, but including the auditors' manually determined voter intent from the audited cards.

The current version uses a single sample to audit all contests. It is possible to refine things to target smaller contests.

In [0]:
from __future__ import division, print_function

import math
import json
import warnings
import numpy as np
import pandas as pd
import csv
import copy

from collections import OrderedDict
from IPython.display import display, HTML

from cryptorandom.cryptorandom import SHA256
from cryptorandom.sample import sample_by_index

from assertion_audit_utils import \
    Assertion, Assorter, CVR, TestNonnegMean, check_audit_parameters, find_margins,\
    find_p_values, find_sample_size, new_sample_size, \
    prep_sample, summarize_status, write_audit_parameters
from dominion_tools import \
    prep_dominion_manifest, sample_from_cvr, write_cards_sampled


# Audit parameters.

* `seed`: the numeric seed for the pseudo-random number generator used to draw sample 
* `replacement`: whether to sample with replacement. If the sample is drawn with replacement, gamma must also be specified.
* `risk_function`: the function to be used to measure risk. Options are `kaplan_markov`,`kaplan_wald`,`kaplan_kolmogorov`,`wald_sprt`,`kaplan_martingale`. Not all risk functions work with every social choice function. `wald_sprt` applies only to plurality contests.
* `g`: a parameter to hedge against the possibility of observing a maximum overstatement. Require $g \in [0, 1)$ for `kaplan_kolmogorov`, `kaplan_markov`, and `kaplan_wald`
* `N_cards`: an upper bound on the number of pieces of paper cast in the contest. This should be derived independently of the voting system. A ballot consists of one or more cards.

----

* `cvr_file`: filename for CVRs (input)
* `manifest_file`: filename for ballot manifest (input)
* `manifest_type`: "STYLE" if the manifest is supposed to list only cards that contain the contests under audit; "ALL" if the manifest contains all cards cast in the election
* `assertion_file`: filename of assertions for IRV contests, in RAIRE format
* `sample_file`: filename for sampled card identifiers (output)
* `mvr_file`: filename for manually ascertained votes from sampled cards (input)
* `log_file`: filename for audit log (output)

----

* `error_rate`: expected rate of 1-vote overstatements. Recommended value $\ge$ 0.001 if there are hand-marked ballots. Larger values increase the initial sample size, but make it more likely that the audit will conclude in a single round if the audit finds errors

* `contests`: a dict of contest-specific data 
    + the keys are unique contest identifiers for contests under audit
    + the values are dicts with keys:
        - `risk_limit`: the risk limit for the audit of this contest
        - `cards_cast`: an upper bound on the number of cast cards that contain the contest
        - `choice_function`: `plurality`, `supermajority`, or `IRV`
        - `n_winners`: number of winners for majority contests. (Multi-winner IRV not supported; multi-winner super-majority is nonsense)
        - `share_to_win`: for super-majority contests, the fraction of valid votes required to win, e.g., 2/3.
        - `candidates`: list of names or identifiers of candidates
        - `reported_winners` : list of identifier(s) of candidate(s) reported to have won. Length should equal `n_winners`.
        - `assertion_file`: filename for a set of json descriptors of Assertions (see technical documentation) that collectively imply the reported outcome of the contest is correct. Required for IRV; ignored for other social choice functions

In [0]:
seed = 72135370570268865044  # use, e.g., 20 rolls of a 10-sided die. Seed doesn't have to be numeric
replacement = False
N_cards = 15428 # From Alaska Ballot manifest
#risk_function = "kaplan_martingale"
#risk_fn = lambda x: TestNonnegMean.kaplan_martingale(x, N=N_cards)[0]
# In the April 2020 AK audit, kaplan_kolmogorov with g=0.1 would have 
#    been more efficient:
risk_function = "kaplan_kolmogorov"
risk_fn = lambda x: TestNonnegMean.kaplan_kolmogorov(x, N=N_cards, g=0.1)
g=0.1


In [0]:
cvr_file = './Data/WY/WY_ballots.raire'
manifest_file = './Data/WY/BallotManifest_WY.xlsx'
manifest_type = 'STYLE'
sample_file = 'sample.csv'
mvr_file = 'mvr.json'
log_file = 'log.json'

In [0]:
error_rate = 0.002      # expect 2 1-vote overstatements per 1000 ballots

In [0]:
# contests to audit. Edit with details of your contest (eg., Contest 339 is the DA race)
contests = {'1':{'risk_limit':0.05,
                     'choice_function':'IRV',
                     'n_winners':1,
                     'candidates':['7','11','1','9','8','5','4','3','12'],
                     'reported_winners' : ['1'],
                     'assertion_file' : './Data/WY/WY_Assertions.json'
                    }
           }

Example of other social choice functions:

> contests =  {'city_council':{'risk_limit':0.05,
                     'choice_function':'plurality',
                     'n_winners':3,
                     'candidates':['Doug','Emily','Frank','Gail','Harry'],
                     'reported_winners' : ['Doug', 'Emily', 'Frank']
                    },
            'measure_1':{'risk_limit':0.05,
                     'choice_function':'supermajority',
                     'share_to_win':2/3,
                     'n_winners':1,
                     'candidates':['yes','no'],
                     'reported_winners' : ['yes']
                    }                  
           }

In [0]:
# read the assertions for the IRV contest
for c in contests:
    if contests[c]['choice_function'] == 'IRV':
        with open(contests[c]['assertion_file'], 'r') as f:
            contests[c]['assertion_json'] = json.load(f)['audits'][0]['assertions']

In [0]:
# construct the dict of dicts of assertions for each contest
all_assertions = Assertion.make_all_assertions(contests)

In [8]:

all_assertions

{'1': {'1 v 4': <assertion_audit_utils.Assertion at 0x7f34052d5550>,
  '1 v 3': <assertion_audit_utils.Assertion at 0x7f34052cac50>,
  '1 v 5': <assertion_audit_utils.Assertion at 0x7f34052ca3c8>,
  '1 v 8': <assertion_audit_utils.Assertion at 0x7f34052ca860>,
  '1 v 9': <assertion_audit_utils.Assertion at 0x7f34052ca780>,
  '1 v 12': <assertion_audit_utils.Assertion at 0x7f34052cf710>,
  '1 v 11': <assertion_audit_utils.Assertion at 0x7f34052cfba8>,
  '1 v 7': <assertion_audit_utils.Assertion at 0x7f34052cf898>}}

## Read the ballot manifest

In [0]:
# special for Primary/Dominion manifest format
manifest = pd.read_excel(manifest_file)

## Read the CVRs 

In [10]:
cvr_input = []
with open(cvr_file) as f:
    cvr_reader = csv.reader(f, delimiter=',', quotechar='"')
    for row in cvr_reader:
        cvr_input.append(row)

print("Read {} rows".format(len(cvr_input)))

Read 15430 rows


In [11]:
# Import CVRs
cvr_list = CVR.from_raire(cvr_input)
print("After merging, there are CVRs for {} cards".format(len(cvr_list)))

After merging, there are CVRs for 15428 cards


In [0]:
# turn RAIRE-style identifiers into Dominion-style by substituting "-" for "_"
for c in cvr_list:
    c.set_id(str(c.id).replace("_","-"))

In [13]:
for i in range(10):
    print(str(cvr_list[i]))

id: 2-1-1 votes: {'1': {'4': 1}} phantom: False
id: 2-1-2 votes: {'1': {'11': 1, '1': 2, '5': 3, '3': 4, '8': 5}} phantom: False
id: 2-1-3 votes: {'1': {'1': 1, '4': 2}} phantom: False
id: 2-1-4 votes: {'1': {'12': 1, '4': 2, '1': 3, '8': 4, '9': 5}} phantom: False
id: 2-1-5 votes: {'1': {'1': 1}} phantom: False
id: 2-1-6 votes: {'1': {'1': 1, '11': 2, '4': 3, '8': 4}} phantom: False
id: 2-1-7 votes: {'1': {'8': 1, '1': 2}} phantom: False
id: 2-1-8 votes: {'1': {'4': 1, '5': 2, '1': 3, '11': 4, '8': 5}} phantom: False
id: 2-1-9 votes: {'1': {'1': 1, '4': 2, '8': 3}} phantom: False
id: 2-1-10 votes: {'1': {'4': 1, '8': 2, '5': 3, '1': 4, '9': 5}} phantom: False


In [14]:
# Check that there is a CVR for every card cast in the contest. If not, add phantoms.

n_cvrs = len(cvr_list)
manifest, manifest_cards, phantom_cards = prep_dominion_manifest(manifest, N_cards, n_cvrs)

manifest

,State,Batch Letter,Tabulator ID,Batch ID,Ballot Count,Transfer Case #,cum_cards
0,Wyoming,A,2,1,100,TC-001,100
1,Wyoming,B,2,2,100,TC-001,200
2,Wyoming,C,2,3,100,TC-001,300
3,Wyoming,D,2,4,100,TC-001,400
4,Wyoming,E,2,5,100,TC-001,500
5,Wyoming,F,2,6,100,TC-001,600
6,Wyoming,G,2,7,100,TC-001,700
7,Wyoming,H,2,8,100,TC-001,800
8,Wyoming,A,2,9,100,TC-007,900
9,Wyoming,B,2,10,100,TC-007,1000


In [15]:
# Create CVRs and MVRs for phantom cards
# If the sample draws a phantom card, these CVRs will be used in the comparison.
# phantom MVRs should be treated as zeros by the Assorter for every contest
phantom_vrs = []
for i in range(phantom_cards):
    phantom_vrs.append(CVR(id='phantom-1-'+str(i+1), votes={}, phantom = True))  # matches expected RAIRE id for parsing later
    
cvr_list = cvr_list + phantom_vrs

print("Created {} phantom records".format(len(phantom_vrs)))

Created 0 phantom records


In [16]:
manifest_cards

15428

In [17]:
# find the mean of the assorters for the CVRs and check whether the assertions are met
min_margin = find_margins(contests, all_assertions, cvr_list)

print("minimum assorter margin {}".format(min_margin))
for c in contests:
    print("margins in contest {}".format(c))
    for a, m in contests[c]['margins'].items():
        print(a, m)

minimum assorter margin 0.38235675395385016
margins in contest 1
1 v 4 0.38235675395385016
1 v 3 0.635079077002852
1 v 5 0.6050038890329272
1 v 8 0.5364272750842625
1 v 9 0.633264194970184
1 v 12 0.6343012704174229
1 v 11 0.5942442312678247
1 v 7 0.6189395903551984


In [0]:
check_audit_parameters(risk_function, g, error_rate, contests)

In [0]:
write_audit_parameters(log_file, seed, replacement, risk_function, g, N_cards, n_cvrs, \
                       manifest_cards, phantom_cards, error_rate, contests)

## Set up for sampling

## Find initial sample size

In [20]:
# find initial sample size
ss_fn = lambda m, r: TestNonnegMean.initial_sample_size(\
                        risk_function=risk_fn, N=N_cards, margin=m,\
                        error_rate=error_rate, alpha=r)
sample_size = find_sample_size(contests, all_assertions, sample_size_function=ss_fn)
print(sample_size)
sample_size = 20
print(sample_size)

17
20


## Draw the first sample

In [21]:
# draw the initial sample
prng = SHA256(seed)
sample = sample_by_index(N_cards, sample_size, prng=prng) # 1-indexed
n_phantom_sample = np.sum([cvr_list[i].phantom for i in sample])
print("The sample includes {} phantom cards.".format(n_phantom_sample))

The sample includes 0 phantom cards.


In [0]:
cvr_sample_lookup, cvr_sample, mvr_phantoms_sample = sample_from_cvr(cvr_list, manifest, sample)

In [0]:
# write the sample
write_cards_sampled(sample_file, cvr_sample_lookup, print_phantoms=False)

## Read the audited sample data

In [0]:
with open(mvr_file) as f:
    mvr_json = json.load(f)

mvr_sample = CVR.from_dict(mvr_json['ballots'])

In [0]:
# add MVRs for phantoms
mvr_sample = mvr_sample + mvr_phantoms_sample

## Find measured risks for all assertions

In [26]:
prep_sample(mvr_sample, cvr_sample)
p_max = find_p_values(contests, all_assertions, mvr_sample, cvr_sample, manifest_type, \
                      risk_function= risk_fn)
print("maximum assertion p-value {}".format(p_max))
done = summarize_status(contests, all_assertions)

maximum assertion p-value 0.027409774202528284
p-values for assertions in contest 1
1 v 4 0.027409774202528284
1 v 3 0.0014207361644163214
1 v 5 0.0020844675993174615
1 v 8 0.0048303092003429415
1 v 9 0.0014543733676772481
1 v 12 0.001435061907128555
1 v 11 0.002385497881384822
1 v 7 0.0017472686526191757

contest 1 AUDIT COMPLETE at risk limit 0.05. Attained risk 0.027409774202528284


In [0]:
# Log the status of the audit 
write_audit_parameters(log_file, seed, replacement, risk_function, g, N_cards, n_cvrs, \
                       manifest_cards, phantom_cards, error_rate, contests)

# How many more cards should be audited?

Estimate how many more cards will need to be audited to confirm any remaining contests. The enlarged sample size is based on:

* cards already sampled
* the assumption that we will continue to see errors at the same rate observed in the sample

In [0]:
# Estimate sample size required to confirm the outcome, if errors continue
# at the same rate as already observed.

new_size, sams = new_sample_size(contests, all_assertions, mvr_sample,\
                                 cvr_sample, manifest_type,\
                                 risk_fn, quantile=0.8, reps=100)
new_size

/Users/stark/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:90: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


114.0

In [0]:
# augment the sample
# reset the seed
prng = SHA256(seed)
old_sample = sample
sample = sample_by_index(N_cards, new_size, prng=prng)  # sampling without replacement
incremental_sample = np.sort(list(set(sample) - set(old_sample)))
n_phantom_sample = np.sum([cvr_list[i].phantom for i in incremental_sample])
print("The incremental sample includes {} phantom cards.".format(n_phantom_sample))

In [0]:
cvr_sample_lookup_new, cvr_sample_new, mvr_phantoms_sample_new = \
                sample_from_cvr(cvr_list, manifest, incremental_sample)
write_cards_sampled(sample_file, cvr_sample_lookup_new, print_phantoms=False)

In [0]:
# mvr_json should contain the complete set of mvrs, including those in previous rounds

with open(mvr_file) as f:
    mvr_json = json.load(f)

mvr_sample = CVR.from_dict(mvr_json['ballots']) 

In [0]:
# compile entire sample
cvr_sample_lookup, cvr_sample, mvr_phantoms_sample = sample_from_cvr(cvr_list, manifest, sample)

In [0]:
# add MVRs for phantoms
mvr_sample = mvr_sample + mvr_phantoms_sample

## Find measured risks for all assertions

In [0]:
prep_sample(mvr_sample, cvr_sample)
p_max = find_p_values(contests, all_assertions, mvr_sample, cvr_sample, manifest_type, \
                      risk_function= risk_fn)
print("maximum assertion p-value {}".format(p_max))
done = summarize_status(contests, all_assertions)

In [0]:
# Log the status of the audit 
write_audit_parameters(log_file, seed, replacement, risk_function, g, N_cards, n_cvrs, \
                       manifest_cards, phantom_cards, error_rate, contests)